# Cálculo de taxa de mortalidade

## Preparação do Notebook

Na célula adiante são feitos procedimentos de preparação do notebook para trabalho com os dados, tais como:
1. mapeamento do diretório em que se encontra o pacote br_demography;
2. importação de bibliotecas necessárias para processamento dos dados;
3. carregametno de project_id para faturamento no Google Cloud, conforme requerido pela biblioteca basedosdados.

In [1]:
# adicionando br_demography to Python Path
import sys 
sys.path.append("../../")

# carregamento de bibliotecas para processamento dos dados
from br_demography  import municipality_deaths as md
import basedosdados as bd
from dotenv import load_dotenv
import os
import pandas as pd # importação de biblioteca de manipulação de dados tabulares
import matplotlib.pyplot as plt # importação de biblioteca de plotagem de gráficos 
import numpy as np # importação de biblioteca para cálculos matemáticos
from scipy.optimize import curve_fit # importação de função a ser utilizada para ajuste de modelo preditivo 
from itertools import product # importação de função voltada para otimização de laços de iteração
import warnings
from itertools import product



# Carregando as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path='../../.env')

# Carregando o project_id para faturamento no Google Cloud
project_id = os.getenv('GOOGLE_CLOUD_PROJECT_ID')

## Carregamento, tratamento e salvamento de microdados sobre nascidos vivos

Com base nos dados do Sistema de Informações sobre Mortalidade - SIM, sao realizados os seguintes processamentos:
1. carregamento do arquivo com os municípios de interesse;
2. _loop_ para cada município de interesse;
3. download dos microdados SIM por consulta SQL à _DataLake_ da Base dos Dados;
4. tratamento dos dados (padronização de faixas etárias decenais, indexação, inserção de valor "0" para coortes com valor nulo, padronização de tipagem, adequação de colunas, etc.);
5. concatenação dos dados de cada município no DataFrae df_deaths
6. salvamento dos no arquivo obitos_rmc_2000_2022.csv

In [2]:
df_mun = pd.read_csv('../../br_demography/source/tab/cod_mun.csv', sep=';', index_col='mun_id')

dict_dfs = dict()

for mun_id, mun in df_mun.itertuples():
    df_deaths_mun = md.query_deaths(mun_id=mun_id, project_id=project_id, start_year=2000, end_year=2022)
    df_deaths_mun = md.standard_age_groups(df=df_deaths_mun, age_group_csv_path='../../br_demography/source/tab/faixas_etarias_censo_2000_2010.csv')
    df_deaths_mun['Município'] = mun
    df_deaths_mun = df_deaths_mun.reset_index().set_index(['Município', 'Sexo','Faixa Etária'])
    
    dict_dfs[mun] = df_deaths_mun

df_deaths = pd.concat(objs=dict_dfs.values())

df_deaths.to_csv(path_or_buf='../../br_demography/results/tab/obitos_rmc_2000_2022.csv', sep=';', decimal=',', encoding='utf-16')

Downloading: 100%|██████████| 627/627 [00:00<00:00, 1918.94rows/s]


### Visualização parcial do DataFrame df_deaths, com microdados de óbitos por coorte

In [3]:
df_deaths.iloc[np.r_[0:2, -2:0]]

Ano                                       2000  2001  2002  2003  2004  2005  \
Município      Sexo      Faixa Etária                                          
Adrianópolis   Feminino  0 a 9 anos          1     1     1     0     0     1   
                         10 a 19 anos        0     1     0     1     1     0   
Doutor Ulysses Masculino 70 a 79 anos        3     4     3     4     2     1   
                         80 anos ou mais     2     3     2     4     2     2   

Ano                                       2006  2007  2008  2009  ...  2013  \
Município      Sexo      Faixa Etária                             ...         
Adrianópolis   Feminino  0 a 9 anos          1     1     2     1  ...     0   
                         10 a 19 anos        0     0     1     2  ...     2   
Doutor Ulysses Masculino 70 a 79 anos        2     5     2     5  ...     3   
                         80 anos ou mais     1     1     6     2  ...     4   

Ano                                       2014  2015  2016  2017  2018  2019  \
Município      Sexo      Faixa Etária                                          
Adrianópolis   Feminino  0 a 9 anos          0     0     0     1     0     2   
                         10 a 19 anos        0     0     0     0     0     0   
Doutor Ulysses Masculino 70 a 79 anos        4     3     1     3     2     0   
                         80 anos ou mais     3     5     4     3     9     7   

Ano                                       2020  2021  2022  
Município      Sexo      Faixa Etária                       
Adrianópolis   Feminino  0 a 9 anos          0     0     1  
                         10 a 19 anos        0     0     0  
Doutor Ulysses Masculino 70 a 79 anos        5     8     2  
                         80 anos ou mais     2     8     3  

[4 rows x 23 columns]

### Carregamento dos dados de população entre 2000 e 2022

In [4]:
df_pop = pd.read_csv(filepath_or_buffer='../../br_demography/results/tab/pop_municipios_rmc_2000_2022_estimativa_intercensitaria.csv', sep=';')
#df_pop[df_pop['Faixa Etária'].isin(values=df_deaths.index)]
df_pop.set_index(keys=['Município', 'Sexo','Faixa Etária'], inplace=True)
df_pop.columns = df_pop.columns.astype(int)
df_pop.iloc[np.r_[0:2, -2:0]]

2000  2001  2002  2003  2004  2005  \
Município      Sexo      Faixa Etária                                          
Adrianópolis   Feminino  0 a 9 anos        704   681   660   639   619   599   
                         10 a 19 anos      823   798   775   752   730   708   
Doutor Ulysses Masculino 70 a 79 anos       79    79    80    81    82    83   
                         80 anos ou mais    22    23    24    26    28    30   

                                          2006  2007  2008  2009  ...  2013  \
Município      Sexo      Faixa Etária                             ...         
Adrianópolis   Feminino  0 a 9 anos        580   562   544   527  ...   485   
                         10 a 19 anos      687   667   647   628  ...   576   
Doutor Ulysses Masculino 70 a 79 anos       84    85    86    87  ...   100   
                         80 anos ou mais    31    34    36    38  ...    46   

                                          2014  2015  2016  2017  2018  2019  \
Município      Sexo      Faixa Etária                                          
Adrianópolis   Feminino  0 a 9 anos        476   468   460   452   444   437   
                         10 a 19 anos      565   555   545   534   524   515   
Doutor Ulysses Masculino 70 a 79 anos      104   108   112   117   122   127   
                         80 anos ou mais    48    49    52    54    56    58   

                                          2020  2021  2022  
Município      Sexo      Faixa Etária                       
Adrianópolis   Feminino  0 a 9 anos        429   422   415  
                         10 a 19 anos      505   496   487  
Doutor Ulysses Masculino 70 a 79 anos      132   137   143  
                         80 anos ou mais    60    63    66  

[4 rows x 23 columns]

## Cálculo de Taxa de Mortalidade

In [5]:
# criação de dataframe com taxa de mortalidade a cada mil habitantes
df_mortalidade = (df_deaths / df_pop).dropna(axis=0) * 1000

df_mortalidade.drop(columns=[2020, 2021, 2022], inplace=True)


## Modelagem das tendências

1. definição de modelos (linear e exponencial)
4. para coortes com tendência geral de crescimento, aplicação de modelo linear;
5. para coortes com tendência geral de decréscimo, modelo exponencial;
6. resultados são armazenados em DataFrame; 
6. plotagem dos resultados;  
6. resultados são armazenados no arquivo taxa_natalidade_projetada_rmc_2022_2045_2.csv.

In [6]:
from scipy.stats import iqr

# Função para o modelo exponencial decrescente
def modelo_exponencial(x, constante, coeficiente):
    return constante * np.exp(coeficiente * (-x))

# Função para o modelo Linear - casos de crescimento
def modelo_linear(x, constante, coeficiente):
    return coeficiente * x + constante

# DataFrame de resultados
resultados = pd.DataFrame(columns=['Município', 'Sexo', 'Faixa Etária','Tipo de Modelo','Constante', 'Coeficiente'])

# Combinações de sexo e faixa etária
df_mortalidade.reset_index(inplace=True)

plt.ioff()

for mun_id, mun in df_mun.itertuples():
    print(mun)

    sexos = df_mortalidade[df_mortalidade['Município'] == mun]['Sexo'].unique()
    faixas_etarias = df_mortalidade[df_mortalidade['Município'] == mun]['Faixa Etária'].unique()

    num_plots = len(sexos) * len(faixas_etarias)
    num_colunas = 3
    num_linhas = (num_plots + num_colunas - 1) // num_colunas

    # Criar subplots para cada combinação de sexo e faixa etária
    fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(15, 18))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)

    for i, (sexo, faixa_etaria) in enumerate(product(sexos, faixas_etarias)):
        if i >= num_plots:
            break  # Sair se todos os subplots já estiverem preenchidos
        
        # Filtrar o DataFrame para a combinação atual
        df_filtro = df_mortalidade[(df_mortalidade['Município'] == mun) & (df_mortalidade['Sexo'] == sexo) & (df_mortalidade['Faixa Etária'] == faixa_etaria)]
        
        # Extrair os dados de anos e mortalidade
        anos = np.array(df_filtro.columns[4:], dtype=int) - 2000
        dados_mortalidade = df_filtro.iloc[0, 4:].values.astype(float)
        anos_todo_intervalo = np.arange(0, 47)

        # Filtrar anos com dados de mortalidade não-zero
        anos_com_dados = anos[dados_mortalidade > 0]
        dados_mortalidade = dados_mortalidade[dados_mortalidade > 0]

        outlier_factor = 1.2

        IQR = iqr(dados_mortalidade)
        lower_bound = np.percentile(dados_mortalidade, 25) - outlier_factor * IQR
        upper_bound = np.percentile(dados_mortalidade, 75) + outlier_factor * IQR

        non_outliers_mask = (dados_mortalidade >= lower_bound) & (dados_mortalidade <= upper_bound)
        anos_com_dados = anos_com_dados[non_outliers_mask]
        dados_mortalidade = dados_mortalidade[non_outliers_mask]


        # Verificar se a tendência é de crescimento ou decrescimento
        coef_angular = np.polyfit(anos_com_dados, dados_mortalidade, 1)[0]
        if coef_angular >= 0:

            # Ajustar o modelo linear aos dados
            try:
                params, _ = curve_fit(modelo_linear, anos_com_dados, dados_mortalidade)
                constante, coeficiente = params
                previsao = modelo_linear(anos_todo_intervalo, constante, coeficiente)
                tipo_modelo = 'Linear'

            except RuntimeError:
                constante, coeficiente = np.nan, np.nan
                previsao = np.nan * anos
        else:
            # Ajustar o modelo exponencial aos dados
            try:
                params, _ = curve_fit(modelo_exponencial, anos_com_dados, dados_mortalidade, maxfev=10000)
                constante, coeficiente = params
                previsao = modelo_exponencial(anos_todo_intervalo, constante, coeficiente)
                tipo_modelo = 'Exponencial'

            except RuntimeError:
                constante, coeficiente = np.nan, np.nan
                previsao = np.nan * anos



        # Criar um DataFrame temporário com os resultados atuais
        df_temp = pd.DataFrame({'Município':[mun],'Sexo': [sexo], 'Faixa Etária': [faixa_etaria],
                                'Tipo de Modelo':[tipo_modelo],'Constante': [constante], 'Coeficiente': [coeficiente]})
        
        # Concatenar o DataFrame temporário ao DataFrame de resultados
        resultados = pd.concat([resultados, df_temp], ignore_index=True)

        
        # Plotar os dados originais e a curva ajustada
        row, col = divmod(i, num_colunas)
        axs[row, col].scatter(anos_com_dados + 2000, dados_mortalidade, label='Dados de mortalidade', marker='o')
        axs[row, col].plot(anos_todo_intervalo + 2000, previsao, label=f'Modelo {tipo_modelo} Ajustado', linestyle='--', color='red')
        axs[row, col].set_xlabel('Ano')
        axs[row, col].set_ylabel('Taxa de Óbitos')
        axs[row, col].set_title(f'Sexo: {sexo} | Faixa Etária: {faixa_etaria}')
        axs[row, col].grid(True)
        axs[row, col].legend()

    # Add a single legend below all the plots in the right corner
#    plt.legend(bbox_to_anchor=(.4, -.7), loc='lower left', borderaxespad=0.)


    fig.suptitle(f'Projeção de Mortalidade por Sexo e Faixa - {mun} \n', fontsize=15, fontweight='bold')
    plt.tight_layout()
#    plt.savefig(f'../../br_demography/results/fig/death_rate/teste/mortalidade_{mun}_non_outliers.svg', format="svg")
    plt.savefig(f'../../br_demography/results/fig/death_rate/teste/mortalidade_{mun}_non_outliers_1dot2IQR.png', dpi=300, bbox_inches='tight')
    plt.close(fig)




import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=RuntimeWarning)


Adrianópolis
Agudos do Sul
Almirante Tamandaré
Araucária
Balsa Nova
Bocaiúva do Sul
Campina Grande do Sul
Campo do Tenente
Campo Largo
Campo Magro
Cerro Azul
Colombo
Contenda
Curitiba
Fazenda Rio Grande
Itaperuçu
Lapa
Mandirituba
Piên
Pinhais
Piraquara
Quatro Barras
Quitandinha
Rio Branco do Sul
Rio Negro
São José dos Pinhais
Tijucas do Sul
Tunas do Paraná
Doutor Ulysses


In [7]:
resultados.iloc[np.r_[0:2, -2:0]]

,Município,Sexo,Faixa Etária,Tipo de Modelo,Constante,Coeficiente
0,Adrianópolis,Feminino,0 a 9 anos,Linear,1.436165,0.048381
1,Adrianópolis,Feminino,10 a 19 anos,Linear,0.878513,0.156693
520,Doutor Ulysses,Masculino,70 a 79 anos,Exponencial,43.826318,0.033518
521,Doutor Ulysses,Masculino,80 anos ou mais,Linear,86.260536,0.614724


### Cálculo da Taxa de Sobrevivência entre 2000 e 2045

In [8]:
new_columns = [*resultados.columns.tolist()]
new_columns.extend(range(2022, 2046))
resultados = resultados.reindex(columns=new_columns, fill_value=0)

for year in resultados.columns[6:].astype(int):
    resultados.loc[:,year] = year-2000

resultados_exp = resultados[resultados['Tipo de Modelo'] == 'Exponencial']
resultados_lin = resultados[resultados['Tipo de Modelo'] == 'Linear']

res = pd.concat(
    objs=[
        1 - modelo_exponencial(resultados_exp.iloc[:, 6:], resultados_exp.Constante.values.reshape((-1,1)), resultados_exp.Coeficiente.values.reshape((-1,1)))/1000
        , 1 - modelo_linear(resultados_lin.iloc[:, 6:], resultados_lin.Constante.values.reshape((-1,1)), resultados_lin.Coeficiente.values.reshape((-1,1)))/1000
        ]
    , axis=0
    
)

resultados = pd.concat(
    objs=[
        resultados_exp.iloc[:,:6]
        , resultados_lin.iloc[:,:6]
        ]
    , axis=0
    
)

resultados = resultados.join(res)

resultados.to_csv(path_or_buf='../../br_demography/results/tab/taxa_sobrevivencia_projetada_pdui_2022_2045.csv', sep=';', decimal=',', encoding='utf-16', index=False)

resultados.iloc[np.r_[0:2, -2:0]]

,Município,Sexo,Faixa Etária,Tipo de Modelo,Constante,Coeficiente,2022,2023,2024,2025,...,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045
3,Adrianópolis,Feminino,30 a 39 anos,Exponencial,2.679227,0.002270,0.997451,0.997457,0.997463,0.997469,...,0.997531,0.997537,0.997542,0.997548,0.997553,0.997559,0.997564,0.997570,0.997575,0.997581
4,Adrianópolis,Feminino,40 a 49 anos,Exponencial,3.496847,0.004617,0.996841,0.996855,0.996870,0.996884,...,0.997039,0.997052,0.997066,0.997079,0.997093,0.997106,0.997120,0.997133,0.997146,0.997159
514,Doutor Ulysses,Masculino,10 a 19 anos,Linear,1.318046,0.016974,0.998309,0.998292,0.998275,0.998258,...,0.998071,0.998054,0.998037,0.998020,0.998003,0.997986,0.997969,0.997952,0.997935,0.997918
521,Doutor Ulysses,Masculino,80 anos ou mais,Linear,86.260536,0.614724,0.900216,0.899601,0.898986,0.898371,...,0.891609,0.890995,0.890380,0.889765,0.889151,0.888536,0.887921,0.887306,0.886692,0.886077
